In [1]:
import torch, torchvision
import sys # Python system library needed to load custom functions
import math # module with access to mathematical functions
import os # for changing the directory

import numpy as np  # for performing calculations on numerical arrays
import pandas as pd  # home of the DataFrame construct, _the_ most important object for Data Science
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt  # allows creation of insightful plots

import librosa

sys.path.append('../../audio_preprocessing')
sys.path.append('../../src')
sys.path.append('../../model_training_utils')


import preprocessing_func_3
from generator_to_dataset_3 import NormalisedDataSet
from gdsc_utils import PROJECT_DIR
import model_training
import model_eval

os.chdir(PROJECT_DIR) # changing our directory to root

In [2]:
df_big_data = pd.read_csv('data/big_processed_data.csv')
df_big_argumented_data = pd.read_csv('data/big_argumentation_data.csv')
df = pd.concat([df_big_data, df_big_argumented_data], ignore_index=True)
df.tail()

,Unnamed: 0,file_path,label
66835,44959,data/big_data_upsample/44959.wav,65
66836,44960,data/big_data_upsample/44960.wav,65
66837,44961,data/big_data_upsample/44961.wav,65
66838,44962,data/big_data_upsample/44962.wav,65
66839,44963,data/big_data_upsample/44963.wav,65


In [3]:
import json

with open('audio_preprocessing/saved_data/upsampled_data_size_128_512.json') as f:
    my_info = json.load(f)

mean, std, class_weights = my_info["mean"], my_info["std"], my_info["weights"]

In [4]:
# def resize_function(output_shape=(40,256)):
#     return torchvision.transforms.Resize(size=output_shape, antialias=False)

# def calculate_melsp(x, n_fft=1024, hop_length=128):
#     stft = np.abs(librosa.stft(x, n_fft=n_fft, hop_length=hop_length))**2
#     log_stft = librosa.power_to_db(stft)
#     melsp = librosa.feature.melspectrogram(S=log_stft,n_mels=40)
#     return melsp

In [5]:
train_df_list = []
val_df_list = []

for i in range(66):
    my_df = df[df["label"] == i]
    current_train_df, current_val_df = train_test_split(my_df, test_size=0.2)
    train_df_list.append(current_train_df)
    val_df_list.append(current_val_df)

df_train = pd.concat(train_df_list, ignore_index=True)
df_val = pd.concat(val_df_list, ignore_index=True)

In [6]:
df_train.shape, df_val.shape

((53441, 3), (13399, 3))

In [7]:
paths, labels = list(df_train["file_path"]), list(df_train["label"])

non_normal_gen = preprocessing_func_3.non_normalised_data_generator(
    paths=paths,
    labels=labels,
#     image_preprocess_fn=resize_function(output_shape=(40,256)),
#     mel_transform_fn=calculate_melsp
)

In [8]:
# mean, std, class_wights = preprocessing_func_3.get_stats_and_class_weights_of_non_normalised_data_gen(
#     non_normal_gen, (128, 512))

In [11]:
mean, std

(-1.092248558998108, 0.793742299079895)

In [12]:
# import json

# def save_as_json(path, description, mean, std, weights):
#     my_dict = {
#         "description": description,
#         "mean": float(mean),
#         "std": float(std),
#         "weights": list(class_wights.astype(float)),
#     }
#     with open(path, 'w') as f:
#         json.dump(my_dict, f)

# save_as_json(
#     "audio_preprocessing/saved_data/upsampled_data_size_128_512.json", 
#     "seconds 1.5, image shape (128,512)", mean, std, class_wights)

In [13]:
train_dataset = NormalisedDataSet(
    non_normalised_data_generator_fn=preprocessing_func_3.non_normalised_data_generator, 
    normalised_data_generator_fn=preprocessing_func_3.normalised_data_generator,
    df=df_train, 
    mean=mean,
    std=std,
    shuffle=True,
#     image_preprocess_fn=resize_function(output_shape=(40,256)),
#     mel_transform_fn=calculate_melsp,
)

val_dataset = NormalisedDataSet(
    non_normalised_data_generator_fn=preprocessing_func_3.non_normalised_data_generator, 
    normalised_data_generator_fn=preprocessing_func_3.normalised_data_generator,
    df=df_val, 
    mean=mean,
    std=std,
    shuffle=False,
#     image_preprocess_fn=resize_function(output_shape=(40,256)),
#     mel_transform_fn=calculate_melsp,
)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=28)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=28)

In [14]:
device = model_training.get_device()

In [15]:
from torchvision.models import resnet34, ResNet34_Weights
import torch.nn as nn
import torch.optim as optim

#resnet_model = resnet34(weights=ResNet34_Weights.DEFAULT)
resnet_model = resnet34()
resnet_model.fc = nn.Linear(512, 66)
resnet_model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
resnet_model = resnet_model.to(device)

In [16]:
optimizer = optim.Adam(resnet_model.parameters(), amsgrad=True)
loss = nn.CrossEntropyLoss()

In [17]:
model_training.training(
    model=resnet_model, 
    optimizer=optimizer, 
    loss_fn=loss, 
    train_dataloader=train_dataloader, 
    val_dataloader=val_dataloader, 
    model_path="models/resnet34", 
    start_epoch=5,
    epochs=500,
    early_stop_thresh=50,
)

End of epoch 6: training accuracy = 98.37%, training loss = 0.05308880211062246, training time taken = 793.74 seconds
End of epoch 6: validation accuracy = 98.52%, validation loss = 0.04734859758787974, validation time taken = 188.64 seconds
End of epoch 7: training accuracy = 98.54%, training loss = 0.04689596940446583, training time taken = 579.60 seconds
End of epoch 7: validation accuracy = 98.83%, validation loss = 0.037738715527538366, validation time taken = 143.75 seconds
End of epoch 8: training accuracy = 98.89%, training loss = 0.03713494312561997, training time taken = 592.32 seconds
End of epoch 8: validation accuracy = 98.47%, validation loss = 0.05446168677000858, validation time taken = 147.34 seconds
End of epoch 9: training accuracy = 99.19%, training loss = 0.02672748032079566, training time taken = 600.84 seconds
End of epoch 9: validation accuracy = 98.77%, validation loss = 0.039639012913303386, validation time taken = 138.91 seconds
End of epoch 10: training accu

KeyboardInterrupt: 

In [18]:
torch.save(resnet_model, 'models/resnet34/resnet34_model_epoch_14.pth')

In [19]:
import preprocessing_func_2

df_big_long_wav = pd.read_csv('data/metadata.csv')
df_val_long_wav = df_big_long_wav[df_big_long_wav["subset"]=="validation"]

def get_df_from_class(class_num, df):
    new_df = df[df["label"] == class_num]
    return new_df

for i in range(66):
    df_val_new = get_df_from_class(i, df_val_long_wav)
    paths, labels = list(df_val_new["path"]), list(df_val_new["label"])
    non_normalised_generator = preprocessing_func_2.non_normalised_data_generator(
        paths=paths, 
        labels=labels,
#         image_preprocess_fn=resize_function(output_shape=(40,256)),
#         mel_transform_fn=calculate_melsp,
    )
    normalised_generator = preprocessing_func_2.normalised_data_generator(
        non_normalised_generator, mean, std)
    print(f"we are now in class {i}")
    final_pred = model_eval.evaluation(resnet_model, normalised_generator)
    print()

we are now in class 0
accuracy = 87.27%
predictions = [0, 43, 0, 43, 29, 28, 43, 0, 10, 43, 43, 43, 43, 29, 43, 43, 43, 43, 43, 28, 43, 43, 43, 28, 0, 43, 43, 29, 43, 43, 29, 29, 28, 43, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 29, 29, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 29, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 29, 0, 29, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

we are now in class 1
accuracy = 90.48%
predictions = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

accuracy = 93.33%
predictions = [20, 20, 20, 20, 20, 20, 20, 49, 49, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20]

we are now in class 21
accuracy = 89.08%
predictions = [45, 45, 45, 45, 4, 4, 4, 4, 4, 4, 4, 4, 64, 64, 64, 64, 64, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 5, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 5, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21]

we are now in class 22
accuracy = 73.33%
predictions = [22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22,

accuracy = 97.64%
predictions = [34, 34, 26, 8, 10, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34]

we are now in class 35
accuracy = 66.67%
predictions = [35, 35, 35, 35, 35, 35, 35, 53, 35, 47, 9, 9, 47, 53, 53, 53, 35, 35, 35, 35, 35, 35, 35, 35]

we are now in class 36
accuracy = 97.22%
predictions = [36, 36, 64, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 

accuracy = 45.45%
predictions = [59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 26, 59, 59, 59, 3, 3, 59, 59, 59, 1, 1, 1, 59, 59, 1, 59, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 56, 1, 1, 1, 1, 1, 1, 3, 3, 59, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 11, 65, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 59, 1, 1, 1, 1, 1, 59, 1, 1, 1, 1, 3, 3, 59, 1, 3, 3, 3, 1, 1, 3, 1, 11, 11, 59, 59, 17, 55, 55, 55, 2, 55, 55]

we are now in class 60
accuracy = 62.50%
predictions = [60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 49, 49, 55, 55, 60, 51, 49, 64, 64, 43, 60, 60, 60]

we are now in class 61
accuracy = 88.08%
predictions = [60, 60, 60, 39, 60, 60, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 6

In [20]:
df_big_long_wav = pd.read_csv('data/metadata.csv')
df_val_long_wav = df_big_long_wav[df_big_long_wav["subset"]=="validation"]


paths, labels = list(df_val_long_wav["path"]), list(df_val_long_wav["label"])
non_normalised_generator = preprocessing_func_2.non_normalised_data_generator(
    paths=paths, 
    labels=labels,
#     image_preprocess_fn=resize_function(output_shape=(40,256)),
#     mel_transform_fn=calculate_melsp,
)
normalised_generator = preprocessing_func_2.normalised_data_generator(non_normalised_generator, mean, std)
model_eval.evaluation(resnet_model, normalised_generator)

accuracy = 79.24%
predictions = [3, 3, 3, 3, 3, 8, 26, 53, 39, 39, 39, 39, 38, 39, 39, 39, 39, 38, 38, 38, 38, 38, 39, 39, 39, 39, 39, 39, 39, 39, 18, 18, 18, 18, 18, 18, 60, 7, 16, 7, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 64, 64, 60, 18, 18, 18, 64, 64, 60, 55, 44, 58, 44, 44, 44, 44, 58, 44, 44, 44, 44, 44, 30, 30, 30, 30, 30, 30, 39, 38, 38, 23, 23, 23, 23, 23, 34, 34, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 13, 39, 1, 1, 1, 26, 26, 26, 26, 39, 7, 7, 7, 26, 26, 13, 26, 26, 39, 39, 7, 7, 7, 26, 41, 41, 26, 26, 26, 39, 7, 7, 7, 41, 48, 23, 23, 23, 23, 23, 24, 24, 24, 24, 24, 24, 30, 30, 30, 30, 30, 64, 44, 44, 44, 44, 44, 61, 44, 61, 44, 44, 44, 44, 65, 64, 55, 60, 64, 37, 37, 37, 17, 7, 7, 7, 17, 17, 17, 17, 17, 17, 17, 17, 17, 16, 16, 16, 59, 59, 59, 59, 59, 59, 59, 59, 16, 59, 59, 16, 16, 16, 56, 16, 16, 16, 17, 7, 7, 7, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 16, 17, 17, 17, 16, 17, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 17, 16, 17, 17, 17, 59, 59, 17, 17,

([3,
  3,
  3,
  3,
  3,
  8,
  26,
  53,
  39,
  39,
  39,
  39,
  38,
  39,
  39,
  39,
  39,
  38,
  38,
  38,
  38,
  38,
  39,
  39,
  39,
  39,
  39,
  39,
  39,
  39,
  18,
  18,
  18,
  18,
  18,
  18,
  60,
  7,
  16,
  7,
  7,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  64,
  64,
  60,
  18,
  18,
  18,
  64,
  64,
  60,
  55,
  44,
  58,
  44,
  44,
  44,
  44,
  58,
  44,
  44,
  44,
  44,
  44,
  30,
  30,
  30,
  30,
  30,
  30,
  39,
  38,
  38,
  23,
  23,
  23,
  23,
  23,
  34,
  34,
  39,
  39,
  39,
  39,
  39,
  39,
  39,
  39,
  39,
  39,
  39,
  39,
  39,
  39,
  39,
  39,
  39,
  39,
  13,
  39,
  1,
  1,
  1,
  26,
  26,
  26,
  26,
  39,
  7,
  7,
  7,
  26,
  26,
  13,
  26,
  26,
  39,
  39,
  7,
  7,
  7,
  26,
  41,
  41,
  26,
  26,
  26,
  39,
  7,
  7,
  7,
  41,
  48,
  23,
  23,
  23,
  23,
  23,
  24,
  24,
  24,
  24,
  24,
  24,
  30,
  30,
  30,
  30,
  30,
  64,
  44,
  44,
  44,
  44,
  44,
  61,
  44,
  61,
  44,
  44,
  44,
  44,
  65,
  64

In [21]:
df_big_long_wav = pd.read_csv('data/metadata.csv')
df_val_long_wav = df_big_long_wav[df_big_long_wav["subset"]=="validation"]
df_val_long_wav.head()

paths, labels = list(df_val_long_wav["path"]), list(df_val_long_wav["label"])

In [22]:
result = model_eval.evaluate_audio_classes([resnet_model], paths, labels, mean, std)

accuracy = 100.00%
predictions = [3, 3, 3, 3, 3]
accuracy = 0.00%
predictions = [8, 26]
accuracy = 100.00%
predictions = [53]
accuracy = 57.14%
predictions = [39, 39, 39, 39, 38, 39, 39, 39, 39, 38, 38, 38, 38, 38]
accuracy = 100.00%
predictions = [39, 39, 39, 39, 39, 39, 39, 39]
accuracy = 100.00%
predictions = [18, 18, 18, 18, 18, 18]
accuracy = 0.00%
predictions = [60]
accuracy = 0.00%
predictions = [7, 16, 7, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6]
accuracy = 100.00%
predictions = [64, 64]
accuracy = 100.00%
predictions = [60]
accuracy = 100.00%
predictions = [18, 18, 18]
accuracy = 100.00%
predictions = [64, 64]
accuracy = 100.00%
predictions = [60]
accuracy = 100.00%
predictions = [55]
accuracy = 16.67%
predictions = [44, 58, 44, 44, 44, 44, 58, 44, 44, 44, 44, 44]
accuracy = 100.00%
predictions = [30, 30, 30, 30, 30, 30]
accuracy = 33.33%
predictions = [39, 38, 38]
accuracy = 100.00%
predictions = [23, 23, 23]
accuracy = 100.00%
predictions = [23, 23]
accuracy = 100.00%
predictions = [34,

accuracy = 94.12%
predictions = [41, 41, 41, 41, 41, 41, 5, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41]
accuracy = 100.00%
predictions = [9, 9, 9]
accuracy = 100.00%
predictions = [9, 9, 9, 9, 9]
accuracy = 100.00%
predictions = [39, 39, 39, 39, 39, 39, 39, 39]
accuracy = 100.00%
predictions = [34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34]
accuracy = 100.00%
predictions = [36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36]
accuracy = 66.67%
predictions = [46, 46, 47, 46, 46, 47]
accuracy = 100.00%
predictions = [26, 26]
accuracy = 100.00%
predictions = [31]
accuracy = 100.00%
predictions = [17, 17]
accuracy = 100.00%
predictions = [42, 42, 42]
accuracy = 100.00%
predictions = [14, 14, 14, 14]
accuracy = 33.33%
predictions = [37, 7, 7]
accuracy = 8

accuracy = 14.29%
predictions = [8, 11, 11, 11, 11, 26, 11]
accuracy = 100.00%
predictions = [20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20]
accuracy = 85.71%
predictions = [27, 12, 12, 12, 12, 12, 12]
accuracy = 100.00%
predictions = [10, 10]
accuracy = 100.00%
predictions = [13]
accuracy = 100.00%
predictions = [61, 61, 61, 61, 61, 61, 61]
accuracy = 100.00%
predictions = [38, 38, 38, 38, 38, 38, 38]
accuracy = 100.00%
predictions = [59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59]
accuracy = 100.00%
predictions = [61, 61, 61, 61, 61]
accuracy = 100.00%
predictions = [38, 38, 38, 38]
accuracy = 100.00%
predictions = [61, 61, 61, 61, 61, 61]
accuracy = 97.67%
predictions = [7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,

accuracy = 100.00%
predictions = [21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21]
accuracy = 33.33%
predictions = [60, 9, 60, 60, 60, 9]
accuracy = 100.00%
predictions = [32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32]
accuracy = 100.00%
predictions = [38, 38, 38]
accuracy = 66.67%
predictions = [2, 65, 65]
accuracy = 0.00%
predictions = [32]
accuracy = 100.00%
predictions = [54, 54]
accuracy = 0.00%
predictions = [16]
accuracy = 100.00%
predictions = [63]
accuracy = 0.00%
predictions = [24, 24, 24]
accuracy = 0.00%
predictions = [38, 38, 38, 38, 55, 55]
accuracy = 100.00%
predictions = [19, 19, 19, 19, 19]
accuracy = 64.38%
predictions = [45, 45, 45, 45, 17, 45, 17, 17, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 56, 45, 45, 45, 45, 45, 45, 45, 45, 56, 56, 45, 56, 45, 56, 56, 56, 56, 56, 

accuracy = 92.31%
predictions = [3, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53]
accuracy = 100.00%
predictions = [37, 37, 37]
accuracy = 100.00%
predictions = [40]
accuracy = 0.00%
predictions = [59]
accuracy = 100.00%
predictions = [63]
accuracy = 100.00%
predictions = [37, 37]
accuracy = 100.00%
predictions = [19, 19]
accuracy = 100.00%
predictions = [14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14]
accuracy = 0.00%
predictions = [16, 38, 12, 12, 12, 12, 12, 12, 12]
accuracy = 100.00%
predictions = [54]
accuracy = 94.74%
predictions = [25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 2, 25, 25]
accuracy = 100.00%
predictions = [11, 11, 11, 11, 11, 11, 11, 11, 11, 11]
accuracy = 100.00%
predictions = [59, 59, 59]
accuracy = 100.00%
predictions = [32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32]
accuracy = 57.14%
predictions = [43, 1, 43, 43, 64, 43, 11, 11, 11, 11, 11, 11, 11, 11]
accuracy = 100.00%
predictions = [40, 40]
accuracy = 100

accuracy = 0.00%
predictions = [24]
accuracy = 100.00%
predictions = [5]
accuracy = 0.00%
predictions = [17, 55, 55, 55, 2, 55, 55]
accuracy = 100.00%
predictions = [29]
accuracy = 100.00%
predictions = [5]
accuracy = 71.43%
predictions = [9, 9, 9, 9, 26, 26, 9]
accuracy = 75.00%
predictions = [26, 9, 9, 9]
accuracy = 100.00%
predictions = [5]
accuracy = 100.00%
predictions = [56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56]
accuracy = 100.00%
predictions = [62, 62, 62]
accuracy = 0.00%
predictions = [38, 38, 38, 38, 38, 38, 38, 38, 38]
accuracy = 78.57%
predictions = [55, 55, 55, 39, 10, 55, 55, 39, 55, 55, 55, 55, 55, 55]
accuracy = 100.00%
predictions = [5]
accuracy = 72.73%
predictions = [9, 9, 9, 9, 10, 38, 9, 9, 9, 9, 38]
accuracy = 100.00%
predictions = [33, 33, 33, 33]
accuracy = 53.12%
predictions = [37, 18, 34, 61, 18, 37, 61, 61, 61, 34, 34, 61, 61, 34, 37, 37, 61, 61, 61, 61, 61, 61, 34, 61, 61, 61, 34, 18, 29, 61, 61, 61, 61, 61, 61, 34, 61, 61, 61, 49, 61, 

In [23]:
df_result = pd.DataFrame.from_dict(result)

df_result.head()

,paths,true_label,predicted_labels,predicted_class_id
0,data/val/Atrapsaltacorticina_GBIF2901504947_IN...,3,"[3, 3, 3, 3, 3]",3
1,data/val/Chorthippusbrunneus_XC751398-dat022-0...,10,"[8, 26]",8
2,data/val/Psaltodaplaga_GBIF3031797565_IN684694...,53,[53],53
3,data/val/Omocestusviridulus_XC752267-dat013-00...,39,"[39, 39, 39, 39, 38, 39, 39, 39, 39, 38, 38, 3...",39
4,data/val/Omocestusviridulus_XC752263-dat012-00...,39,"[39, 39, 39, 39, 39, 39, 39, 39]",39


In [24]:
np.mean(df_result["true_label"].values == df_result["predicted_class_id"].values)

0.8134715025906736

In [25]:
for i in range(66):
    df_class = df_result[df_result["true_label"] == i]
    class_accuracy = 100 * np.mean(df_class["true_label"].values == df_class["predicted_class_id"].values) 
    print(f"we are now in class {i} with accuracy {class_accuracy:.2f}%")

we are now in class 0 with accuracy 90.00%
we are now in class 1 with accuracy 100.00%
we are now in class 2 with accuracy 100.00%
we are now in class 3 with accuracy 66.67%
we are now in class 4 with accuracy 100.00%
we are now in class 5 with accuracy 100.00%
we are now in class 6 with accuracy 44.44%
we are now in class 7 with accuracy 100.00%
we are now in class 8 with accuracy 12.50%
we are now in class 9 with accuracy 86.96%
we are now in class 10 with accuracy 86.96%
we are now in class 11 with accuracy 100.00%
we are now in class 12 with accuracy 87.50%
we are now in class 13 with accuracy 75.00%
we are now in class 14 with accuracy 100.00%
we are now in class 15 with accuracy 75.00%
we are now in class 16 with accuracy 60.00%
we are now in class 17 with accuracy 63.16%
we are now in class 18 with accuracy 66.67%
we are now in class 19 with accuracy 100.00%
we are now in class 20 with accuracy 100.00%
we are now in class 21 with accuracy 71.43%
we are now in class 22 with accur